# <img src="https://upload.wikimedia.org/wikipedia/commons/6/60/NISAR_artist_concept.jpg" width=400 align="left"/><br><br><br><br>



<img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/NISAR_Mission_Logo.png" width=400 align="left"/><br><br><br><br><br>



# NASA ISRO Synthetic Aperture Radar Mission
## Combined Algorithm Theoretical Basis Document and Jupyter Notebook for <br> *Classification of Wetland Inundation Extent*


Authors: Bruce Chapman, Paul Siqueira

Date: February 15, 2022

Last updated: December 2024, Brandi Downs

### Summary
This notebook describes the ATBD for generating a wetland inundation product from NISAR time series data stacks. First, the images of the multi-temporal sequence must be well radiometrically calibrated relative to each other, to a higher precision than perhaps required through routine standard calibration of the NISAR imagery. This optional calibration step examines distributed targets that are expected to be unchanged or minimally changed in brightness over a set time span of  an image sequence. With NISAR’s 240 km swath width, it is reasonably assumed that a statistically large area, A<sub>ni</sub>, will not be inundated (or otherwise changing) during any of the 2n observations surrounding the image to be calibrated and classified. These areas will be identified through use of a priori wetlands mask and partly through image segmentation or other methods over the 2n images. <br>
A set of classes will be identified from a multitemporal average of a subset of images including:

- Inundated vegetation (presumption: dominated by double bounce scatter in HH channel) 
- Open water (presumption: low specular scattering in both channels)
- Not inundated (presumption: brighter specular scatter, volume scattering)
- Not classified (presumption: pixels do not align with the scattering model, or no data)

These classes are selected based on calibrated threshold values for the radar backscatter and other metrics. In addition, this same multi-temporal image sequence allows the algorithm to include a more sensitive change detection component for improved robustness. Change detection will allow for refinement within the multitemporal image sequence for change of class during the image sequence that may be more robust than simply classifying the image backscatter and backscatter ratio values.  


### Use environment:
`ecosystems_atbd`

## Step 1: Read in GCOV Data

In [ ]:
import os
import h5py
import numpy as np
import s3fs
import matplotlib.pyplot as plt
import warnings
from yaml import safe_load, safe_dump
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from datetime import datetime
from pathlib import Path
from osgeo import gdal, osr
import rasterio
import rioxarray as rxr

In [ ]:
# get filenames nested within S3 bucket directories
calval_site = 'Yucatan_Lake'
gcov_url = ("s3://nisar-st-data-ondemand/ALOS2_processed/Yucatan_Lake_Louisiana/51/630/GCOV/")
s3 = s3fs.S3FileSystem()
gcov_dirs = ['s3://' + k['Key'] + '/' for k in s3.listdir(gcov_url)]

fstr = '.h5'
stats_str = 'STATS'
gcov_files = []
for k in gcov_dirs:
    temp = ['s3://' + f['Key'] for f in s3.listdir(k)]
    for j in temp:
        if fstr in j and stats_str not in j:
            gcov_files.append(j)

# sort by date
gcov_files = sorted(gcov_files)

num_files = len(gcov_files)

print(num_files, 'files')
print(*gcov_files, sep='\n')

In [ ]:
# read in all h5 files
h5_files = [h5py.File(s3.open(k, "rb")) for k in gcov_files]
gcov_filenames = [k.filename.strip('<').strip('>').split('/')[-1] for k in h5_files]
gcov_dates = [ (str(k.split('_')[11][:4]) + '-' + str(k.split('_')[11][4:6]) + '-' + str(k.split('_')[11][6:8])) for k in gcov_filenames]
gcov_filenames

In [ ]:
# List all groups and datasets in H5 file
def print_all_objs(name, obj):
    print(obj)

#h5_files[0].visititems(print_all_objs)

In [ ]:
# Get xy coordinates, epsg, and backscatter data
# This cell may take a few minutes to run if there are many files in h5_files

ds_x = [f['science']['LSAR']['GCOV']['grids']['frequencyA']['xCoordinates'][()] for f in h5_files]
ds_y = [f['science']['LSAR']['GCOV']['grids']['frequencyA']['yCoordinates'][()] for f in h5_files]
ds_epsg = [f['science']['LSAR']['GCOV']['grids']['frequencyA']['projection'][()].item() for f in h5_files]
ds_HHHH = [f['science']['LSAR']['GCOV']['grids']['frequencyA']['HHHH'][()] for f in h5_files] 
ds_HVHV = [f['science']['LSAR']['GCOV']['grids']['frequencyA']['HVHV'][()] for f in h5_files] 

[f.close() for f in h5_files];

In [ ]:
# make sure x and y arrays are the same for each scene
# if they are - keep just 1 copy
# if not, print warning

x_equal, y_equal, epsg_equal = True, True, True

for i,k in enumerate(ds_x):
    if i == 0:
        continue
    if ( (k != ds_x[0]).any() ):
        x_equal = False
        warning_text = 'x array index ' + str(i) + ' unequal'
        warnings.warn(warning_text)

for i,k in enumerate(ds_y):
    if i == 0:
        continue
    if ( (k != ds_y[0]).any() ):
        y_equal = False
        warning_text = 'x array index ' + str(i) + ' unequal'
        warnings.warn(warning_text)

for i,k in enumerate(ds_epsg):
    if i == 0:
        continue
    if ( k != ds_epsg[0] ):
        epsg_equal = False
        warning_text = 'epsg array index ' + str(i) + ' unequal'
        warnings.warn(warning_text)


print('x arrays equal:', x_equal)
print('y arrays equal:', y_equal)
print('epsg values equal:', epsg_equal)

if x_equal:
    ds_x = ds_x[0]
if y_equal:
    ds_y = ds_y[0]
if epsg_equal:
    ds_epsg = ds_epsg[0]

In [ ]:
# plot the GCOV images

fig, axs = plt.subplots(num_files, 2, figsize=(12,num_files*5))
cbar_shrink = 0.6

for k in range(num_files):

    im1 = axs[k][0].imshow(ds_HHHH[k], vmin=0, vmax=0.5, cmap='gray')
    title_str = gcov_dates[k] + ', ' + 'HHHH'
    axs[k][0].set_title(title_str)
    fig.colorbar(im1, ax=axs[k][0], shrink=cbar_shrink)

    im2 = axs[k][1].imshow(ds_HVHV[k], vmin=0, vmax=0.1, cmap='gray')
    title_str = gcov_dates[k] + ', ' + 'HVHV'
    axs[k][1].set_title(title_str)
    fig.colorbar(im2, ax=axs[k][1], shrink=cbar_shrink)



In [ ]:
# Compute ratio, product, and sum
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    ds_ratio = [ds_HHHH[k]/ds_HVHV[k] for k in range(num_files)]
    ds_product = [ds_HHHH[k]*ds_HVHV[k] for k in range(num_files)]
    ds_sum = [ds_HHHH[k]+ds_HVHV[k] for k in range(num_files)]

In [ ]:
# Plot ratio, product, sum
fig, axs = plt.subplots(num_files, 3, figsize=(14,num_files*5))
cbar_shrink = 0.6

for k in range(num_files):

    im1 = axs[k][0].imshow(ds_ratio[k], vmin=1, vmax=12, cmap='gray')
    title_str = gcov_dates[k] + ', ' + 'ratio'
    axs[k][0].set_title(title_str)
    fig.colorbar(im1, ax=axs[k][0], shrink=cbar_shrink)

    im2 = axs[k][1].imshow(ds_product[k], vmin=0, vmax=0.05, cmap='gray')
    title_str = gcov_dates[k] + ', ' + 'product'
    axs[k][1].set_title(title_str)
    fig.colorbar(im2, ax=axs[k][1], shrink=cbar_shrink)

    im3 = axs[k][2].imshow(ds_sum[k], vmin=0, vmax=0.5, cmap='gray')
    title_str = gcov_dates[k] + ', ' + 'sum'
    axs[k][2].set_title(title_str)
    fig.colorbar(im3, ax=axs[k][2], shrink=cbar_shrink)

## Step 2: Read in the classification thresholds

In [ ]:
# Read in contents of yaml configuration file
with s3.open('nisar-st-data-ondemand/wetlands_config/wetland_calibration_parameters.yaml', 'rb') as f:
    config_doc = safe_load(f)

class_thresh = config_doc['runconfig']['calval_sites'][calval_site]
class_thresh

From the classification thresholds for this calval site, Yucatan Lake, printed above, we see that only `inun_veg_single_class` and `open_water` are used. For `open_water`, the sum is used, not the product. We will save each threshold next.

In [ ]:
# Save thresholds
comb_method = 'sum'  # combination method is either product or sum; in this case, it's sum
th = {}

# inundated vegetation (iv)
th['iv_hh_max'] = class_thresh['inun_veg_single_class']['HH_max']
th['iv_hh_min'] = class_thresh['inun_veg_single_class']['HH_min']
th['iv_ratio_max'] = class_thresh['inun_veg_single_class']['ratio_max']
th['iv_ratio_min'] = class_thresh['inun_veg_single_class']['ratio_min']

# open water (ow)
if comb_method == 'sum':
    th['ow_comb_max'] = class_thresh['open_water']['sum_max']
    th['ow_comb_min'] = class_thresh['open_water']['sum_min']
elif comb_method == 'product':
    th['ow_comb_max'] = class_thresh['open_water']['product_max']
    th['ow_comb_min'] = class_thresh['open_water']['product_min']    
else:
    raise Exception("Invalid combination method") 

th

## Step 3: Classify the GCOV data

In [ ]:
# First classify open water, then classify remaining (non-open water) pixels as inun veg or not inundated

np.seterr(invalid='ignore')
nisar_classified_20m = []

for k in range(num_files):
    temp = np.zeros(ds_HHHH[k].shape, dtype=np.int8)

    # set all valid data pixels to 1
    if comb_method == 'sum':
        ds_comb = ds_sum[k].copy()
    else:
        ds_comb = ds_product[k].copy()
    idx = ds_comb > 0
    temp[idx] = 1
    numpx = np.sum(temp.ravel())

    # set open water pixels to 2
    idx = (ds_comb > th['ow_comb_min']) & (ds_comb <= th['ow_comb_max'])
    temp[idx] = 2

    # set inundate vegetation pixels to 3
    idx = (ds_HHHH[k] >= th['iv_hh_min']) & (ds_HHHH[k] <= th['iv_hh_max']) & \
           (ds_ratio[k] >= th['iv_ratio_min']) & (ds_ratio[k] <= th['iv_ratio_max']) & \
           (temp != 2)
    temp[idx] = 3

    # print('Open water: ' + str(np.round(1000*np.sum(temp.ravel()==2)/numpx)/10) + '%')
    # print('Inun veg I: ' + str(np.round(1000*np.sum(temp.ravel()==3)/numpx)/10) + '%')
    # print('Not inun: '  + str(np.round(1000*np.sum(temp.ravel()==1)/numpx)/10) + '%')

    nisar_classified_20m.append(temp)


In [ ]:
# plot classified images

# set up colormaps
c_white = (255, 255, 255)
c_lightblue = (66, 233, 245)
c_darkblue = (21, 27, 115)
c_gray = (236, 236, 238)

colors = [c_white, c_gray, c_darkblue, c_lightblue]
colors2 = []
for k in colors:
    colors2.append(tuple(np.array(k)/255)) 
cmap = LinearSegmentedColormap.from_list('cmap_class', colors2, N=4)

fig, axs = plt.subplots(num_files, 1, figsize=(6,num_files*5))
cbar_shrink = 0.6
cbar_ticks = [3/8, 9/8, 15/8, 21/8]
cbar_labels = ['no data','not inun','open water','inun veg']
# cbar_ticks = [4/10, 12/10, 20/10, 28/10, 36/10]  # for 2 inun veg classes
# cbar_label = ['no data','not inun','open water','inun veg I','inun veg II']  # for 2 inun veg classes

for k in range(num_files):

    im = axs[k].imshow(nisar_classified_20m[k], vmin=0, vmax=3, cmap=cmap, interpolation='nearest')
    axs[k].set_title(gcov_dates[k])
    cbar = plt.colorbar(im, ax=axs[k], shrink=cbar_shrink)    
    cbar.set_ticks(cbar_ticks)
    cbar.set_ticklabels(cbar_labels, fontsize=10)    


## Step 4: Aggregate to 1 Hectare

In [ ]:
# output results as geotiff using rasterio

class_dir = Path(os.getcwd()) / 'nisar_classifications' / calval_site
Path(class_dir).mkdir(parents=True, exist_ok=True)

classification_20m_filenames = []
classification_1ha_filenames = []

for k in range(num_files):

    # create 20m geotiffs to use as inputs for gdal Warp
    filename_20m = str(class_dir / ('nisar_classified_20m_' + datetime.today().strftime('%Y%m%d') + '_gcov_' + "".join(gcov_dates[k].split('-')) + \
                      '_' + calval_site + '.tif'))
    classification_20m_filenames.append(filename_20m)
    meta = {'driver': 'GTiff', 
            'dtype': 'float32', 
            'nodata': None, 
            'width': ds_x.shape[0], 
            'height': ds_y.shape[0], 
            'count': 1, 
            'crs': rasterio.CRS.from_epsg(ds_epsg), 
            'transform': rasterio.Affine(ds_x[1] - ds_x[0], 0.0, ds_x[0] - ((ds_x[1] - ds_x[0])/2), 0.0, ds_y[1] - ds_y[0], ds_y[0] - ((ds_y[1] - ds_y[0])/2)),
            'tiled': False, 
            'interleave': 'band'}
    with rasterio.open(filename_20m, 'w', **meta) as dst:
        dst.write(nisar_classified_20m[k],indexes=1)    

    L3_filename_1ha = str(class_dir / ('nisar_classified_1ha_' + datetime.today().strftime('%Y%m%d') + '_gcov_' + "".join(gcov_dates[k].split('-')) + \
                      '_' + calval_site + '.tif'))
    classification_1ha_filenames.append(L3_filename_1ha)
    gdal.Warp(L3_filename_1ha, filename_20m, xRes=100, yRes=-100, resampleAlg=gdal.GRA_Mode, format="COG")

    # optional: remove 20m files
    os.remove(filename_20m)
    

In [ ]:
# read in the 1 ha data

nisar_classified_1ha = []

ds = rxr.open_rasterio(classification_1ha_filenames[0])
ds_1ha_x = ds.x
ds_1ha_y = ds.y
ds_1ha_epsg = ds.rio.crs.to_authority()[1]

for k in classification_1ha_filenames:
    ds = rxr.open_rasterio(k)
    nisar_classified_1ha.append(ds.to_numpy().squeeze().astype(np.int8))
   

In [ ]:
# plot 1-ha classified images

# set up colormaps
c_white = (255, 255, 255)
c_lightblue = (66, 233, 245)
c_darkblue = (21, 27, 115)
c_gray = (236, 236, 238)

colors = [c_white, c_gray, c_darkblue, c_lightblue]
colors2 = []
for k in colors:
    colors2.append(tuple(np.array(k)/255)) 
cmap = LinearSegmentedColormap.from_list('cmap_class', colors2, N=4)

fig, axs = plt.subplots(num_files, 1, figsize=(6,num_files*5))
cbar_shrink = 0.6
cbar_ticks = [3/8, 9/8, 15/8, 21/8]
cbar_labels = ['no data','not inun','open water','inun veg']
# cbar_ticks = [4/10, 12/10, 20/10, 28/10, 36/10]  # for 2 inun veg classes
# cbar_label = ['no data','not inun','open water','inun veg I','inun veg II']  # for 2 inun veg classes

for k in range(num_files):

    im = axs[k].imshow(nisar_classified_1ha[k], vmin=0, vmax=3, cmap=cmap, interpolation='nearest')
    axs[k].set_title(gcov_dates[k])
    cbar = plt.colorbar(im, ax=axs[k], shrink=cbar_shrink)    
    cbar.set_ticks(cbar_ticks)
    cbar.set_ticklabels(cbar_labels, fontsize=10)    
